In [1]:
%load_ext autoreload
%autoreload 2
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from takehome.tools import query_vector_db_articles, get_smiles_from_pubchem, predict_energy_from_smiles
title = "Targeted therapy in advanced non-small cell lung cancer: current advances and future trends."
title2 = "Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors."

In [3]:
## CODE TO DELETE PINECONE:
import os
from pinecone import Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("pubmed")

index.delete(delete_all=True)

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'Date': 'Sun, 09 Jun 2024 18:54:00 GMT', 'Content-Type': 'application/json', 'Content-Length': '55', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1', 'x-pinecone-request-id': '1027437128989184151', 'x-envoy-upstream-service-time': '1', 'server': 'envoy'})
HTTP response body: {"code":5,"message":"Namespace not found","details":[]}


In [4]:
# completion llm  
from langchain_cohere import ChatCohere
import os

llm = ChatCohere(model="command-r-plus",temperature=0.0,max_tokens=4096)

In [5]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

# Preamble
preamble = """
You are an expert who answers the user's question with the most relevant datasource.
You must answer in a precise and concise manner.
You are equipped with a vectordatabase about scientific articles,
a tool to get the SMILES string of a given molecule name,
and a tool to predict the energy of a molecule from its SMILES string.
"""

# Prompt template
prompt = ChatPromptTemplate.from_template("{input}")
# Create the ReAct agent
agent = create_cohere_react_agent(
   llm=llm,
   tools=[query_vector_db_articles,get_smiles_from_pubchem, predict_energy_from_smiles],
   prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent,
                               tools=[query_vector_db_articles,get_smiles_from_pubchem, predict_energy_from_smiles],
                               verbose=True,
                               handle_parsing_errors=True,)

memory = ChatMessageHistory(session_id="test-session")

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [6]:
response = agent_with_chat_history.invoke(
    {"input": "What disease are they discussing in the paper '{}'?".format(title)},
    config={"configurable": {"session_id": "<foo>"}},
)
print(response['output'])

Parent run 9689418b-6616-4133-aaaa-1b802034265e not found for run 14bb503a-1230-453a-b427-d93d68c1dbe7. Treating as a root run.




> Entering new AgentExecutor chain...

I will search for the paper 'Targeted therapy in advanced non-small cell lung cancer: current advances and future trends' and then write an answer based on the information I find.
{'tool_name': 'query_vector_db_articles', 'parameters': {'query': 'Targeted therapy in advanced non-small cell lung cancer: current advances and future trends', 'metadata': {'title': 'Targeted therapy in advanced non-small cell lung cancer: current advances and future trends'}}}
[Document(page_content='Majeed\xa0 et\xa0al. J Hematol Oncol          (2021) 14:108  \nhttps://doi.org/10.1186/s13045-021-01121-2\nREVIEW\nTargeted therapy in\xa0advanced non-small cell \nlung cancer: current advances and\xa0future trends\nUmair Majeed, Rami Manochakian, Yujie Zhao and Yanyan Lou*  \nAbstract \nLung cancer remains the leading cause of cancer-related mortality in both men and women in the US and worldwide. \nNon-small cell lung cancer is the most common variety accounting for 84

In [7]:
response = agent_with_chat_history.invoke(
    {"input": "Summarize the main takeaways from the paper."},
    config={"configurable": {"session_id": "<foo>"}},
)
print(response['output'])

Parent run d8d45b8e-e729-48a2-8c7a-aeb4158d0351 not found for run 3db07edc-d85e-419e-a58d-f30048a003d6. Treating as a root run.




> Entering new AgentExecutor chain...

I will read the paper and summarise the main takeaways.
{'tool_name': 'query_vector_db_articles', 'parameters': {'query': 'Targeted therapy in advanced non-small cell lung cancer: current advances and future trends.', 'metadata': {'title': 'Targeted therapy in advanced non-small cell lung cancer: current advances and future trends.'}}}
[Document(page_content='Majeed\xa0 et\xa0al. J Hematol Oncol          (2021) 14:108  \nhttps://doi.org/10.1186/s13045-021-01121-2\nREVIEW\nTargeted therapy in\xa0advanced non-small cell \nlung cancer: current advances and\xa0future trends\nUmair Majeed, Rami Manochakian, Yujie Zhao and Yanyan Lou*  \nAbstract \nLung cancer remains the leading cause of cancer-related mortality in both men and women in the US and worldwide. \nNon-small cell lung cancer is the most common variety accounting for 84% of the cases. For a subset of patients with \nactionable mutations, targeted therapy continues to provide durable respon

In [8]:
response = agent_with_chat_history.invoke(
    {"input": "Can you list the protein targets they highlight in the paper?"},
    config={"configurable": {"session_id": "<foo>"}},
)
print(response['output'])


Parent run bd88bcad-c465-4ba1-980d-af4c919f8e03 not found for run f7a4232b-939c-42e5-a8aa-a35d47bf1cdc. Treating as a root run.




> Entering new AgentExecutor chain...

I will read the paper to find the protein targets highlighted in it.
{'tool_name': 'query_vector_db_articles', 'parameters': {'query': 'protein targets', 'metadata': {'title': 'Targeted therapy in advanced non-small cell lung cancer: current advances and future trends.'}}}
[Document(page_content='Page 20 of 20 Majeed\xa0et\xa0al. J Hematol Oncol          (2021) 14:108 \n•\n fast, convenient online submission\n •\n  thorough peer review by experienced researchers in your ﬁeld\n• \n rapid publication on acceptance\n• \n support for research data, including large and complex data types\n•\n  gold Open Access which fosters wider collaboration and increased citations \n maximum visibility for your research: over 100M website views per year •\n  At BMC, research is always in progress.\nLearn more biomedcentral.com/submissionsReady to submit y our researc h Ready to submit y our researc h  ?  Choose BMC and benefit fr om: ?  Choose BMC and benefit fr o

In [9]:
response = agent_with_chat_history.invoke(
    {"input": "Can you list the small molecule drugs they highlight in the paper?"},
    config={"configurable": {"session_id": "<foo>"}},
)
print(response['output'])

Parent run d1810cbf-9082-40c7-8e3e-64d3064621da not found for run caac612a-77da-4cad-a4fb-02810cf02f67. Treating as a root run.




> Entering new AgentExecutor chain...

I will read the paper to find the small molecule drugs highlighted in it.
{'tool_name': 'query_vector_db_articles', 'parameters': {'query': 'small molecule drugs', 'metadata': {'title': 'Targeted therapy in advanced non-small cell lung cancer: current advances and future trends.'}}}
[Document(page_content='Page 15 of 20\n Majeed\xa0et\xa0al. J Hematol Oncol          (2021) 14:108 \n \nTable 13 Current ongoing early phase I/II trials with miscellaneous target inhibitors\nDrug name Mechanism of action Clinical trial (phase) Study design Disease\nERBB2(HER2) inhibitors\nAdo-trastuzumab emtansine HER2-targeted antibody–drug \nconjugateNCT03784599 (1/2) Combination with osimertinib EGFR mutant advanced NSCLC\nMobocertinib (TAk-788) TKI targeting EGFR ex20ins \nmutationsNCT02716116 (1/2) Combination with chemo -\ntherapyEGFR and HER2 exon 20 inser -\ntion in advanced NSCLC\nPoziotinib EGFR, HER2 and HER 4 inhibi-\ntorsNCT03066206 (2) Monotherapy EGFR 

In [10]:
response = agent_with_chat_history.invoke(
    {"input": "Can you find the SMILES string of the first small molecule you found?"},
    config={"configurable": {"session_id": "<foo>"}},
)
print(response['output'])

Parent run e04b7ada-b4b8-468e-a92f-a33df56e5629 not found for run 7c034fef-7ad5-4480-80db-d0f5605b8a60. Treating as a root run.




> Entering new AgentExecutor chain...

I will search for the SMILES string of Osimertinib.
{'tool_name': 'get_smiles_from_pubchem', 'parameters': {'name': 'Osimertinib'}}
CN1C=C(C2=CC=CC=C21)C3=NC(=NC=C3)NC4=C(C=C(C(=C4)NC(=O)C=C)N(C)CCN(C)C)OCRelevant Documents: 0
Cited Documents: 0
Answer: The SMILES string for Osimertinib is: CN1C=C(C2=CC=CC=C21)C3=NC(=NC=C3)NC4=C(C=C(C(=C4)NC(=O)C=C)N(C)CCN(C)C)OC.
Grounded answer: The SMILES string for Osimertinib is: <co: 0>CN1C=C(C2=CC=CC=C21)C3=NC(=NC=C3)NC4=C(C=C(C(=C4)NC(=O)C=C)N(C)CCN(C)C)OC</co: 0>.

> Finished chain.
The SMILES string for Osimertinib is: CN1C=C(C2=CC=CC=C21)C3=NC(=NC=C3)NC4=C(C=C(C(=C4)NC(=O)C=C)N(C)CCN(C)C)OC.


In [11]:
response = agent_with_chat_history.invoke(
    {"input": "Can you predict its energy value?"},
    config={"configurable": {"session_id": "<foo>"}},
)
print(response['output'])

Parent run 032b0eeb-573d-42be-a2ab-462024848d45 not found for run a8b30c2a-1e4b-4729-b94e-06f1d39dbcd5. Treating as a root run.




> Entering new AgentExecutor chain...

I will use the 'predict_energy_from_smiles' tool to predict the energy value of the SMILES string for Osimertinib.
{'tool_name': 'predict_energy_from_smiles', 'parameters': {'smiles': 'CN1C=C(C2=CC=CC=C21)C3=NC(=NC=C3)NC4=C(C=C(C(=C4)NC(=O)C=C)N(C)CCN(C)C)OC'}}
Prediction for CN1C=C(C2=CC=CC=C21)C3=NC(=NC=C3)NC4=C(C=C(C(=C4)NC(=O)C=C)N(C)CCN(C)C)OC: -11.39196263310017
-11.39196263310017Relevant Documents: 0
Cited Documents: 0
Answer: The predicted energy value of Osimertinib is -11.39196263310017.
Grounded answer: The predicted energy value of Osimertinib is <co: 0>-11.39196263310017</co: 0>.

> Finished chain.
The predicted energy value of Osimertinib is -11.39196263310017.
